In [36]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import aesara


In [37]:
aesara.__file__

'/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/aesara/__init__.py'

In [38]:
aesara.__version__

'2.5.1'

In [52]:
aesara.config.optimizer="None"

ERROR! Session/line number was not unique in database. History logging moved to new session 635


In [54]:
aesara.config.exception_verbosity="high"

In [53]:
aesara.config.optimizer

'None'

ERROR! Session/line number was not unique in database. History logging moved to new session 636


In [55]:
aesara.config.exception_verbosity

'high'

In [46]:
aesara.config.print_active_device

True

In [2]:
data = pd.read_csv("bladderc.csv")

In [3]:
data.head(20)

,time,observed,group
0,0,0,0
1,1,0,0
2,4,0,0
3,7,0,0
4,10,0,0
5,6,1,0
6,14,0,0
7,18,0,0
8,5,1,0
9,12,1,0


So if we oberved the tumor, we want to right-censor the data. If not, we don't.

In [30]:
data["censored"] = 0.0
data.loc[data["observed"] == 0, "censored"] = data["time"]
data.loc[data["observed"] == 1, "censored"] = np.inf

y = data["time"].to_numpy(copy=True)
x = data["group"].to_numpy(copy=True)
censored = data["censored"].to_numpy(copy=True)

In [49]:
y = y.astype(float)

In [50]:
x = x.astype(float)

In [8]:
y

array([ 0.,  1.,  4.,  7., 10.,  6., 14., 18.,  5., 12., 23., 10.,  3.,
        3.,  7.,  3., 26.,  1.,  2., 25., 29., 29., 29., 28.,  2.,  3.,
       12., 32., 34., 36., 29., 37.,  9., 16., 41.,  3.,  6.,  3.,  9.,
       18., 49., 35., 17.,  3., 59.,  2.,  5.,  2.,  1.,  1.,  5.,  9.,
       10., 13.,  3.,  1., 18., 17.,  2., 17., 22., 25., 25., 25.,  6.,
        6.,  2., 26., 38., 22.,  4., 24., 41., 41.,  1., 44.,  2., 45.,
        2., 46., 49., 50.,  4., 54., 38., 59.])

In [27]:
censored

array([ 0.,  1.,  4.,  7., 10., inf, 14., 18., inf, inf, 23., inf, inf,
       inf, inf, inf, 26., inf, inf, inf, 29., 29., 29., inf, inf, inf,
       inf, 32., 34., 36., inf, 37., inf, inf, 41., inf, inf, inf, inf,
       inf, 49., inf, inf, inf, 59., inf, inf, inf,  1.,  1., inf,  9.,
       10., 13., inf, inf, 18., inf, inf, inf, 22., 25., 25., 25., inf,
       inf, inf, inf, 38., inf, inf, inf, 41., 41., inf, 44., inf, 45.,
       inf, 46., 49., 50., inf, 54., inf, 59.])

ERROR! Session/line number was not unique in database. History logging moved to new session 634


In [28]:
censored == y

array([ True,  True,  True,  True,  True, False,  True,  True, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False,  True,  True,  True, False, False, False, False,
        True,  True,  True, False,  True, False, False,  True, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False,  True,  True, False,  True,  True,  True,
       False, False,  True, False, False, False,  True,  True,  True,
        True, False, False, False, False,  True, False, False, False,
        True,  True, False,  True, False,  True, False,  True,  True,
        True, False,  True, False,  True])

In [67]:
censored > y

array([False, False, False, False, False,  True, False, False,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
       False, False, False,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True,  True, False,
        True,  True,  True, False, False,  True, False, False, False,
        True,  True, False,  True,  True,  True, False, False, False,
       False,  True,  True,  True,  True, False,  True,  True,  True,
       False, False,  True, False,  True, False,  True, False, False,
       False,  True, False,  True, False])

In [68]:
censored < y

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [72]:
len(x)

86

In [66]:
# right-censored model
# inits = {'beta0': np.array(.1), 'beta1': np.array(.1)}

with pm.Model() as m:
    beta0 = pm.Cauchy("beta0", alpha=-2, beta=2)
    beta1 = pm.Cauchy("beta1", alpha=-2, beta=2)
    
    λ = aesara.tensor.exp(beta0 + beta1 * x)
    
    #λ_prop = exp(beta0 + beta1 * x)    
    #λ = switch(le(λ_prop, 1), 1, λ_prop)
    
    #obs_latent = pm.Exponential("ol", lam=λ, observed=y)
    obs_latent = pm.Exponential.dist(lam=λ+1000)
        
    likelihood = pm.Censored(
        "likelihood", obs_latent, lower=None, upper=censored, observed=y
    )

    #mu0 = pm.Deterministic("mu0", exp(-beta0))
    #mu1 = pm.Deterministic("mu1", exp(-beta0 - beta1))

    #mu_diff = pm.Deterministic("mu_diff", mu1 - mu0)

    #H_prob = pm.Deterministic("H_prob", switch(ge(mu_diff, 0), 1, 0))

    trace = pm.sample(
        3000,
        chains=4,
        tune=1000,
        cores=4,
        init="adapt_diag",
        return_inferencedata=True,
        step=[pm.NUTS(target_accept=.9)]
    )

az.summary(trace, hdi_prob=0.9)

SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'beta0': array(-2.), 'beta1': array(-2.)}

Initial evaluation results:
{'beta0': -1.84, 'beta1': -1.84, 'likelihood': -inf}

In [ ]:
az.summary(trace, hdi_prob=0.9)